# Data Structures

- Authors: Marc Shapiro, Zeb Engberg
- Date: 2023-04-14
- `pycontrails`: v0.39.6

Overview data structures of `pycontrails`.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/contrailcirrus/2023-04-pycontrails-workshop/blob/main/notebooks/02-Data-Structures.ipynb)

In [ ]:
!pip install pycontrails

# VectorDataset

> This is primarily used as a base class for `GeoVectorDataset` and `Flight`

Structure to hold 1D arrays of consistent size in a dictionary.
Similar to a [`pandas.DataFrame`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html), but more performant.

In [1]:
import numpy as np

from pycontrails import VectorDataset

In [2]:
vector = VectorDataset({
    "a": np.arange(0, 4),
    "b": np.linspace(1, 100, 4),
})
vector

VectorDataset [2 keys x 4 length, 0 attributes]
	Keys: a, b
	Attributes:

You can pass in scalar attributes (`attrs`) to annotate to the dataset

In [3]:
vector = VectorDataset({
    "a": np.arange(0, 4),
    "b": np.linspace(1, 100, 4),
}, attrs={"version": 1})
vector

VectorDataset [2 keys x 4 length, 1 attributes]
	Keys: a, b
	Attributes:
	version             1

Data and attributes are accessed via `data` and `attrs` properties, respectively

In [4]:
print(vector.data)
print(vector.attrs)

{'a': array([0, 1, 2, 3]), 'b': array([  1.,  34.,  67., 100.])}
{'version': 1}


# GeoVectorDataset

> Implements `VectorDataset`

Base class to hold 1D geospatial arrays of consistent size.

GeoVectorDataset is required to have geospatial coordinate keys defined as:

- `latitude`: WGS84, 
- `longitude`: WGS84 
- `altitude`: meters
- `time`: np.datetime64[ns]
- `level`: hPa

Each spatial variable is expected to have "float32" or "float64" `dtype`.
The time variable is expected to have "datetime64[ns]" `dtype`.

Use the attribute `attr["crs"]` to specify coordinate reference system
using [PROJ](https://proj.org/) or [EPSG](https://epsg.org/home.html>) syntax.

In [5]:
import pandas as pd
import numpy as np

from pycontrails import GeoVectorDataset

In [6]:
geovector = GeoVectorDataset({
    "latitude": np.arange(0, 4),
    "longitude": np.linspace(1, 100, 4),
    "altitude": np.linspace(1000, 8000, 4),
    "time": pd.date_range(
                  start="2022-03-10 00",
                  end="2022-03-10 05",
                  periods=4)
}, attrs={"version": 1})
geovector

GeoVectorDataset [4 keys x 4 length, 2 attributes]
	Keys: latitude, longitude, altitude, time
	Attributes:
	time                [2022-03-10 00:00:00, 2022-03-10 05:00:00]
	longitude           [1.0, 100.0]
	latitude            [0.0, 3.0]
	altitude            [1000.0, 8000.0]
	version             1
	crs                 EPSG:4326

# Flight

> Implements `GeoVectorDataset`

A single flight trajectory.

The `Flight` class has many useful utilites.
See the [02-Flight.ipynb](02-Flight.ipynb) for more details.

In [7]:
import pandas as pd
import numpy as np

from pycontrails import Flight

In [8]:
flight = Flight({
    "latitude": np.linspace(38.8, 38.7, 5),
    "longitude": np.linspace(-77, -77.2, 5),
    "altitude": np.linspace(9000, 9000, 5),
    "time": pd.date_range(
                  start="2022-03-01 00:50:00",
                  end="2022-03-01 00:54:00",
                  periods=5)
}, attrs={
    "flight_id": "AC1234",
    "aircraft_type": "A320",
})
flight

Flight [4 keys x 5 length, 3 attributes]
	Keys: latitude, longitude, altitude, time
	Attributes:
	time                [2022-03-01 00:50:00, 2022-03-01 00:54:00]
	longitude           [-77.2, -77.0]
	latitude            [38.7, 38.8]
	altitude            [9000.0, 9000.0]
	flight_id           AC1234
	aircraft_type       A320
	crs                 EPSG:4326

# Meteorology

## MetDataset

Meteorological dataset with multiple variables.

Composition around [xarray.Dataset](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.html#xarray.Dataset) that enforces certain variables and dimensions:

- `latitude`: WGS84, 
- `longitude`: WGS84 
- `level`: hPa, or `altitude`: meters
- `time`: np.datetime64[ns]

See the [02-Meteorology.ipynb](02-Meteorology.ipynb) for more details.

In [9]:
import numpy as np
import xarray as xr

from pycontrails import MetDataset

In [10]:
# create a random number generator
rng = np.random.default_rng(seed=2020)

ds = xr.Dataset(
    {
        "random": (["longitude", "latitude", "level", "time"], rng.random((20, 15, 4, 5))),
        "ones": (["longitude", "latitude", "level", "time"], np.ones((20, 15, 4, 5))),
    },
    coords={
        "longitude": np.arange(-100, -80, 1.0),
        "latitude": np.arange(30, 45, 1.0),
        "level": np.arange(100, 500, 100),
        "time": pd.date_range(
                  start="2022-03-01 00:00:00",
                  end="2022-03-01 05:00:00",
                  periods=5),
    },
)
met = MetDataset(ds)
met

MetDataset with data:

<xarray.Dataset>
Dimensions:       (longitude: 20, latitude: 15, level: 4, time: 5)
Coordinates:
  * longitude     (longitude) float64 -100.0 -99.0 -98.0 ... -83.0 -82.0 -81.0
  * latitude      (latitude) float64 30.0 31.0 32.0 33.0 ... 41.0 42.0 43.0 44.0
  * level         (level) float64 100.0 200.0 300.0 400.0
  * time          (time) datetime64[ns] 2022-03-01 ... 2022-03-01T05:00:00
    air_pressure  (level) float64 1e+04 2e+04 3e+04 4e+04
    altitude      (level) float64 1.618e+04 1.178e+04 9.164e+03 7.185e+03
Data variables:
    random        (longitude, latitude, level, time) float64 0.4683 ... 0.6394
    ones          (longitude, latitude, level, time) float64 1.0 1.0 ... 1.0 1.0

## MetDataArray

Meteorological data array of single variable.

Thin composition around [`xr.DataArray`](https://xarray.pydata.org/en/stable/user-guide/data-structures.html#dataarray) to enforce certain variables and dimensions.

In [11]:
import numpy as np
import xarray as xr

from pycontrails import MetDataArray

In [12]:
rng = np.random.default_rng(seed=2020)

# construct an xarray DataArray with coordinate labels for data
da = xr.DataArray(
    name="random",
    data=rng.random((20, 15, 4, 5)),
    dims=["longitude", "latitude", "level", "time"],
    coords={
        "longitude": np.arange(-100, -80, 1.0),
        "latitude": np.arange(30, 45, 1.0),
        "level": np.arange(100, 500, 100),
        "time": pd.date_range(
                  start="2022-03-01 00:00:00",
                  end="2022-03-01 05:00:00",
                  periods=5),
    },
)
da
met = MetDataArray(da)
met

MetDataArray with data:

<xarray.DataArray 'random' (longitude: 20, latitude: 15, level: 4, time: 5)>
array([[[[4.68307543e-01, 5.14342231e-01, 8.63988281e-01,
          7.19386871e-01, 3.33497882e-01],
         [8.81663616e-01, 5.18664864e-01, 5.23219417e-01,
          7.22388696e-01, 4.46782561e-01],
         [8.50357081e-01, 6.83936263e-01, 6.65900069e-01,
          9.46717074e-01, 7.53638850e-01],
         [9.17964165e-02, 8.26090137e-01, 5.55186777e-01,
          2.31152608e-01, 9.82958913e-01]],

        [[5.84613086e-01, 4.26518507e-01, 8.45336094e-01,
          5.07850470e-01, 7.10216629e-01],
         [1.37352673e-01, 4.84569777e-01, 8.89744202e-01,
          9.52976849e-01, 3.76043941e-02],
         [9.54223457e-01, 8.98512171e-01, 4.14081276e-01,
          4.20221061e-01, 6.91400341e-02],
         [4.21756867e-01, 4.93848391e-01, 1.51696284e-01,
          9.41137061e-01, 1.88116189e-01]],

        [[1.90065507e-02, 9.44932088e-01, 9.93219943e-01,
          6.81811974e-01, 7.84335294e-01],
...
         [8.92843751e-01, 4.07684237e-01, 3.05168469e-01,
          1.73576037e-01, 3.34321840e-01]],

        [[7.61263799e-01, 9.91563651e-01, 9.47543921e-01,
          6.68735461e-01, 2.71486792e-01],
         [2.05356394e-01, 7.42337468e-01, 2.70578729e-01,
          7.15520327e-01, 7.30134504e-01],
         [6.68886804e-01, 5.15651993e-01, 8.40987212e-01,
          5.37939968e-01, 3.71399439e-01],
         [1.67243220e-01, 9.76645528e-01, 3.08876242e-01,
          8.19142068e-01, 9.69391866e-01]],

        [[4.44617360e-01, 5.02274334e-01, 2.32060554e-01,
          3.54429329e-01, 1.36471633e-01],
         [6.30717612e-01, 2.40955809e-01, 2.54390888e-01,
          7.20384123e-01, 3.42038201e-01],
         [8.20903975e-01, 6.73148542e-01, 8.67869039e-01,
          4.79726457e-01, 8.34115517e-01],
         [3.59797874e-01, 7.61370564e-01, 4.16116300e-01,
          7.46702020e-01, 6.39419576e-01]]]])
Coordinates:
  * longitude     (longitude) float64 -100.0 -99.0 -98.0 ... -83.0 -82.0 -81.0
  * latitude      (latitude) float64 30.0 31.0 32.0 33.0 ... 41.0 42.0 43.0 44.0
  * level         (level) float64 100.0 200.0 300.0 400.0
  * time          (time) datetime64[ns] 2022-03-01 ... 2022-03-01T05:00:00
    air_pressure  (level) float64 1e+04 2e+04 3e+04 4e+04
    altitude      (level) float64 1.618e+04 1.178e+04 9.164e+03 7.185e+03